# Grab Dimensions Data

https://app.dimensions.ai/discover/publication

This notebook handles the downloads of Dimensions data.


In [ ]:
import pandas as pd
import numpy as np
import json

## Load Journals

In [ ]:
origin_folder = 'SUSTC_Journals'

In [ ]:
journals_file = f'{origin_folder}/WOS期刊汇总_DimentionsSearch.csv'
ori_journals = pd.read_csv(journals_file, usecols=['search title', 'search id'])
ori_journals = ori_journals.dropna()

In [ ]:
len(ori_journals)

In [ ]:
start_index = 0
end_index = 38
sep_len = 100

In [ ]:
ori_journals

## Request Functions

This part contains functions we need to fetch the web data and should also handle the exceptions while fetching here.

In [ ]:
import requests
import json

from requests import ConnectionError, ReadTimeout

def grab_from_url_content(url):
    headers = {'Accept': '* / *',
               'Accept-Language': 'zh-TW, zh; q=0.9, en-US; q=0.8, en; q=0.7, zh-CN; q=0.6',
               'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3610.2 Safari/537.36'
               }
    rescontent = ''
    try:
        res = requests.get(url, headers=headers, timeout=10)
        rescontent = res.text
    except ConnectionError as ce:
        print('ConnectionError: ' + str(ce))
        return grab_from_url_content(url)
    except ReadTimeout as rte:
        print('ReadTimeout: ' + str(rte))
        return grab_from_url_content(url)

    return rescontent



## Parser

We will get the html content from the url which is not listed as we want it be, so we need parser to parse them into listed data, in json form.

In [ ]:
from html.parser import HTMLParser

class DimensionsHTMLParser(HTMLParser):
    articles = []
    in_article = False
    is_next_trigger = False
    next_trigger_url = ''

    def handle_starttag(self, tag, attrs):
        if tag == 'article':
            self.in_article = True
        
        if self.in_article:
            for attr in attrs:
                if attr[0] == 'data-doc':
                    self.articles.append(json.loads(attr[1]))
                    break

        if tag == 'a':
            for attr in attrs:
                if (attr[0] == 'class') and (attr[1] == 'nextPage-trigger'):
                    self.is_next_trigger = True
                if self.is_next_trigger and (attr[0] == 'href'):
                    self.next_trigger_url = f'https://app.dimensions.ai{attr[1]}&{search_params}'
                    break
        return
                    
    def handle_endtag(self, tag):
        if tag == 'article':
            self.in_article = False
        self.is_next_trigger = False
        return

    def handle_data(self, data):
        pass

    def handle_comment(self, data):
        pass

    def handle_entityref(self, name):
        pass

    def handle_charref(self, name):
        pass

    def handle_decl(self, data):
        pass
    
parser = DimensionsHTMLParser()

## Grab Data

The data fetching starts from here.

In [ ]:
%%time

if end_index > int(len(ori_journals) / sep_len):
    end_index = int(len(ori_journals) / sep_len)
for l in range(start_index, end_index):
    parser.articles = []

    journals_index = str(1 + l * sep_len) + '-' + str((l + 1) * sep_len)
    start = int(journals_index.split('-')[0])
    end = int(journals_index.split('-')[1])
    if end < len(ori_journals):
        journals = ori_journals[start - 1:end]
    else:
        journals = ori_journals[start - 1:]
    print(journals_index)

    for index, row in journals.iterrows():
        journal_search = row['search id']
        # This is very important for data fetching, generally you get url in the form as 'https://app.dimensions.ai/discover/publication?and_facet_for=2209&and_facet_for=2202&or_facet_source_title=jour.1082997', and search_params is just the substring after '?'.
        # Here is where we need to change for each search.
        search_params = f'or_facet_year=2014&or_facet_year=2015&or_facet_year=2016&or_facet_year=2017&or_facet_year=2018&or_facet_source_title={journal_search}'
        grab_url = f'https://app.dimensions.ai/discover/publication.contents.html?{search_params}'
        print(grab_url)

        parser.feed(grab_from_url_content(grab_url))
        next_trigger_anchor = ''
        i = 0
        while parser.next_trigger_url != next_trigger_anchor:
            if i % 10 == 0:
                print(str(i))
            i+=1
            next_trigger_anchor = parser.next_trigger_url
            parser.feed(grab_from_url_content(parser.next_trigger_url))
    
    print(len(parser.articles))
    with open(f"{origin_folder}/articles_wos_{journals_index}.json", "w") as dump_f:
        dump_f.write(json.dumps(parser.articles))

In [ ]:
origin_folder = origin_folder + '/articles_wos'

In [ ]:
%%time
# for l in range(2, int(len(ori_journals) / sep_len)):
for journals_index in ['1-200', '3801-3900']:
    # journals_index = str(1 + l * sep_len) + '-' + str((l + 1) * sep_len)
    with open(f'{origin_folder}/articles_wos_{journals_index}.json') as f:
        articles = json.load(f)

    columns_extract = ['SO', 'SO_id', 'doi', 'dimensions_id', 'title', 'abstract', 'authors', 'references', 'pub_date', 'affiliations_json', 'dimensions_cited']
    df_articles = pd.DataFrame(columns=columns_extract)

    for article in articles:
        if 'doi' not in article:
            continue
        if article['doi'] in df_articles['doi']:
            continue
        df_article = pd.DataFrame(columns=columns_extract)
        df_article['SO'] = [article['source_title']]
        df_article['SO_id'] = [article['source_title_id']]
        df_article['doi'] = [article['doi']]
        df_article['dimensions_id'] = [article['id']]
        df_article['title'] = [article['title']]
        df_article['abstract'] = [article['abstract']]
        if 'author_list' in article:
            df_article['authors'] = [article['author_list']]
        if 'cited_dimensions_ids' in article:
            df_article['references'] = [len(article['cited_dimensions_ids'])]
        if 'pub_date' in article:
            df_article['pub_date'] = [article['pub_date']]
        if 'affiliations_json' in article:
            df_article['affiliations_json'] = [article['affiliations_json']]
        df_article['dimensions_cited'] = [article['times_cited']]
        df_articles = df_articles.append(df_article, ignore_index=True)

    df_articles.to_csv(f'{origin_folder}/articles_wos_{journals_index}.csv', index=False)

In [ ]:
origin_folder = origin_folder + '/articles_abs'

In [ ]:
%%time
for l in range(start_index, int(len(ori_journals) / sep_len)):
    journals_index = str(1 + l * sep_len) + '-' + str((l + 1) * sep_len)
    if journals_index == '801-900':
        journals_index = '801-1100'
    with open(f'{origin_folder}/articles_{journals_index}.json') as f:
        articles = json.load(f)

    columns_extract = ['SO', 'SO_id', 'doi', 'dimensions_id', 'title', 'abstract', 'authors', 'references', 'pub_date', 'affiliations_json', 'dimensions_cited']
    df_articles = pd.DataFrame(columns=columns_extract)

    for article in articles:
        if 'doi' not in article:
            continue
        if article['doi'] in df_articles['doi']:
            continue
        df_article = pd.DataFrame(columns=columns_extract)
        df_article['SO'] = [article['source_title']]
        df_article['SO_id'] = [article['source_title_id']]
        df_article['doi'] = [article['doi']]
        df_article['dimensions_id'] = [article['id']]
        df_article['title'] = [article['title']]
        df_article['abstract'] = [article['abstract']]
        if 'author_list' in article:
            df_article['authors'] = [article['author_list']]
        if 'cited_dimensions_ids' in article:
            df_article['references'] = [len(article['cited_dimensions_ids'])]
        if 'pub_date' in article:
            df_article['pub_date'] = [article['pub_date']]
        if 'affiliations_json' in article:
            df_article['affiliations_json'] = [article['affiliations_json']]
        df_article['dimensions_cited'] = [article['times_cited']]
        df_articles = df_articles.append(df_article, ignore_index=True)

    df_articles.to_csv(f'{origin_folder}/articles_{journals_index}.csv', index=False)

In [ ]:
article

# Affiliations Arrange

In [ ]:
df_articles = pd.read_csv('SUSTC_Journals/articles_all/article_dimensions.csv', usecols=['doi', 'dimensions_id', 'affiliations_json'], nrows=600000)

In [ ]:
df_articles

In [ ]:
%%time

df_researchers = pd.DataFrame()
df_affiliations = pd.DataFrame()

for index, item in df_articles.iterrows():
    if index % 2000 == 0:
        print(index)

    doi = item['doi']
    json_str = item['affiliations_json']
    if pd.isna(json_str):
        continue
    for sub in json.loads(json_str):
        df_researcherx = pd.DataFrame()
        df_affiliationx = pd.DataFrame()

        df_researcherx['doi'] = doi

        if 'first_name' in sub:
            df_researcherx['first_name'] = [sub['first_name']]
        if 'last_name' in sub:
            df_researcherx['last_name'] = [sub['last_name']]
        if 'orcid' in sub:
            df_researcherx['orcid'] = [sub['orcid']]
        if 'current_organization_id' in sub:
            df_researcherx['current_organization_id'] = [sub['current_organization_id']]
        if 'researcher_id' in sub:
            df_researcherx['researcher_id'] = [sub['researcher_id']]
        if 'affiliations' in sub:
            df_researcherx['affiliations_num'] = [len(sub['affiliations'])]
        if 'raw_affiliation' in sub:
            df_researcherx['raw_affiliation'] = [json.dumps(sub['raw_affiliation'])]

        df_researchers = df_researchers.append(df_researcherx, ignore_index=True)

        for subsub in sub['affiliations']:
            if 'id' in subsub:
                df_affiliationx['affiliation_id'] = [subsub['id']]
            if 'name' in subsub:
                df_affiliationx['name'] = [subsub['name']]
            if 'city' in subsub:
                df_affiliationx['city'] = [subsub['city']]
            if 'city_id' in subsub:
                df_affiliationx['city_id'] = [subsub['city_id']]
            if 'country' in subsub:
                df_affiliationx['country'] = [subsub['country']]
            if 'country_code' in subsub:
                df_affiliationx['country_code'] = [subsub['country_code']]
            if 'state' in subsub:
                df_affiliationx['state'] = [subsub['state']]
            if 'state_code' in subsub:
                df_affiliationx['state_code'] = [subsub['state_code']]

            df_affiliations = df_affiliations.append(df_affiliationx, ignore_index=True)

df_researchers.to_csv('SUSTC_Journals/articles_all/article_researchers_1.csv', index=False)
df_affiliations.to_csv('SUSTC_Journals/articles_all/article_affiliations_1.csv', index=False)
